<a href="https://colab.research.google.com/github/suajder/MyFinalProject/blob/master/Maestro_Ema_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed   openpyxl ta statsmodels mplfinance dash
!pip install dash-bootstrap-components tqdm
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
from tqdm import tqdm
import dash
from dash import dcc, html, callback_context
from dash.dependencies import Input, Output, State, ALL
import dash_bootstrap_components as dbc
import ta

# TradingView'e giriş yapmadan bağlan
tv = TvDatafeed()

# Türkiye piyasasındaki tüm sembolleri almak
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# EMA hesaplaması için gün sayıları
EMA_DAYS = [5, 13, 21, 34, 55, 89, 144, 200, 233]

# Hisse verilerini alma ve EMA hesaplaması yapma
def get_hisse_data(hisse):
    try:
        df = tv.get_hist(hisse, exchange='BIST', interval=Interval.in_daily, n_bars=244)
        if df is not None:
            df.index = pd.to_datetime(df.index)
            return df
        else:
            return None
    except Exception as e:
        print(f"Hata: {e}")
        return None


# Tüm hisseler için verileri al ve EMA hesapla
hisse_data = {}
for hisse in tqdm(Hisseler, desc="Veri alınıyor", unit="hisse"):
    data = get_hisse_data(hisse)
    if data is not None:
        hisse_data[hisse] = data

# Crossover kontrol fonksiyonu
def crossover(series1, series2):
    return (series1.iloc[-2] < series2.iloc[-2]) and (series1.iloc[-1] > series2.iloc[-1])

# Tarama kurallarını uygulama
def tarama_kurallari(df, period):
    sonuc = {}
    if len(df) < 30:
        for ema in EMA_DAYS:
            sonuc[f'Yakın_EMA_{ema}'] = False
            sonuc[f'RETEST_{ema}'] = False
            sonuc[f'Yukarı_Kesme_{ema}'] = False
            sonuc[f'Aşağı_Kesme_{ema}'] = False
        return sonuc

    # Günlük veriden hesaplanan EMA'lar
    for ema in EMA_DAYS:
        df[f'EMA_{ema}'] = ta.trend.ema_indicator(df['close'], window=ema)

    # Farklı periyotlar için yeniden örnekleme
    if period == 'günlük':
        data = df.copy()
        for ema in EMA_DAYS:
            data[f'EMA_{ema}'] = ta.trend.ema_indicator(data['close'], window=ema)

    elif period == 'haftalık':
        data = df.resample('W').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
        for ema in EMA_DAYS:
            data[f'EMA_{ema}'] = ta.trend.ema_indicator(data['close'], window=ema)

    elif period == 'aylık':
        data = df.resample('M').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
        for ema in EMA_DAYS:
            data[f'EMA_{ema}'] = ta.trend.ema_indicator(data['close'], window=ema)

    # Aylık ve haftalık işlemlerde EMA hesaplamaları doğru yapılmış mı kontrol ediliyor
    for ema in EMA_DAYS:
        if f'EMA_{ema}' not in data:
            continue

        close_price = data['close'].iloc[-1]  # Son kapanış fiyatı
        previous_close = data['close'].iloc[-2]  # Önceki kapanış fiyatı
        ema_value = data[f'EMA_{ema}'].iloc[-1]  # Son EMA değeri
        previous_ema_value = data[f'EMA_{ema}'].iloc[-2]  # Önceki EMA değeri

        # Fiyat ve EMA değerlerini yazdır
        print(f"{period.capitalize()} Fiyat: {close_price}, EMA: {ema_value}")

        # Yakın EMA kontrolü
        sonuc[f'Yakın_EMA_{ema}'] = abs(close_price - ema_value) / close_price < 0.02

        # Retest kontrolü: Fiyat daha önce EMA'nın altında iken şimdi EMA'nın üzerine çıkmış mı
        sonuc[f'RETEST_{ema}'] = (previous_close < previous_ema_value) and (close_price > ema_value)

        # Yukarı Kesme durumu: Fiyat EMA'yı yukarıdan aşağıya kesti mi
        sonuc[f'Yukarı_Kesme_{ema}'] = crossover(data['close'], data[f'EMA_{ema}'])

        # Aşağı Kesme durumu: Fiyat EMA'yı aşağıdan yukarıya kesti mi
        sonuc[f'Aşağı_Kesme_{ema}'] = (previous_close > previous_ema_value) and (close_price < ema_value)

    return sonuc


# Dash uygulaması oluşturma
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Arayüz düzeni
app.layout = html.Div([
    html.H1("Hisse Tarama Modülü"),
    html.Hr(),
    dcc.Dropdown(
        id='period-secim',
        options=[
            {'label': 'Günlük', 'value': 'günlük'},
            {'label': 'Haftalık', 'value': 'haftalık'},
            {'label': 'Aylık', 'value': 'aylık'}
        ],
        value='günlük',
        placeholder="Periyodu seçin"
    ),
    html.Div(id='sonuc-table', style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px'}),
])
# Sonuçları güncelleme
import os

@app.callback(
    Output('sonuc-table', 'children'),
    [Input('period-secim', 'value')]
)
def update_table(selected_period):
    try:
        tarama_sonuclari = {hisse: tarama_kurallari(df, selected_period) for hisse, df in hisse_data.items()}

        # Excel için DataFrame oluşturma
        rows = []
        for hisse, sonuc in tarama_sonuclari.items():
            row = {'Hisse': hisse}
            row.update(sonuc)
            rows.append(row)

        df_sonuc = pd.DataFrame(rows)
        df_sonuc = df_sonuc.replace(False, "-")
        dosya_adi = f"tarama_sonuclari_{selected_period}.xlsx"
        df_sonuc.to_excel(dosya_adi, index=False)

        print(f"Sonuçlar '{dosya_adi}' dosyasına kaydedildi.")

        # UI kısmı aynı kalabilir
        table_children = []
        for ema in EMA_DAYS:
            hisse_list = []
            retest_list = []
            yukari_kesme_list = []
            for hisse, sonuc in tarama_sonuclari.items():
                if sonuc.get(f'Yakın_EMA_{ema}', False):
                    hisse_list.append(hisse)
                if sonuc.get(f'RETEST_{ema}', False):
                    retest_list.append(hisse)
                if sonuc.get(f'Yukarı_Kesme_{ema}', False):
                    yukari_kesme_list.append(hisse)

            if hisse_list or retest_list or yukari_kesme_list:
                table_children.append(
                    html.Div(children=[
                        html.Div(f'Yakın_EMA_{ema}', style={'fontSize': '14px', 'cursor': 'pointer'},
                                 n_clicks=0, id={'type': 'toggle', 'index': f'Yakın_EMA_{ema}'}),
                        html.Div(id={'type': 'results', 'index': f'Yakın_EMA_{ema}'}, style={'display': 'none'},
                                 children=[
                                     html.Ul([html.Li(hisse) for hisse in hisse_list], style={'paddingLeft': '10px'})
                                 ]),
                        html.Div(f'Retest_{ema}', style={'fontSize': '14px', 'cursor': 'pointer'},
                                 n_clicks=0, id={'type': 'toggle', 'index': f'Retest_{ema}'}),
                        html.Div(id={'type': 'results', 'index': f'Retest_{ema}'}, style={'display': 'none'},
                                 children=[
                                     html.Ul([html.Li(hisse) for hisse in retest_list], style={'paddingLeft': '10px'})
                                 ]),
                        html.Div(f'Yukarı_Kesme_{ema}', style={'fontSize': '14px', 'cursor': 'pointer'},
                                 n_clicks=0, id={'type': 'toggle', 'index': f'Yukarı_Kesme_{ema}'}),
                        html.Div(id={'type': 'results', 'index': f'Yukarı_Kesme_{ema}'}, style={'display': 'none'},
                                 children=[
                                     html.Ul([html.Li(hisse) for hisse in yukari_kesme_list], style={'paddingLeft': '10px'})
                                 ])
                    ])
                )

        if not table_children:
            return html.Div("Hiçbir hisse koşulları karşılamıyor.", style={'fontSize': '14px', 'marginTop': '20px'})

        return table_children

    except Exception as e:
        print(f"Update table error: {e}")
        return html.Div("Veri güncellenirken bir hata oluştu.", style={'color': 'red'})


@app.callback(
    Output({'type': 'results', 'index': ALL}, 'style'),
    Input({'type': 'toggle', 'index': ALL}, 'n_clicks'),
)
def toggle_results(n_clicks):
    if not n_clicks:
        raise dash.exceptions.PreventUpdate

    # Hangi başlıkların tıklandığını kontrol et
    return [{'display': 'none' if clicks % 2 == 0 else 'block'} for clicks in n_clicks]

if __name__ == '__main__':
    app.run(debug=True)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-4ve2aobx
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-4ve2aobx
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Veri alınıyor:  79%|███████▉  | 477/601 [03:57<01:01,  2.01hisse/s]ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol
Veri alınıyor: 100%|██████████| 601/601 [04:58<00:00,  2.01hisse/s]


<IPython.core.display.Javascript object>